<a href="https://colab.research.google.com/github/kavyajeetbora/foursquare_ai/blob/master/notebooks/04_POI_to_PMTiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up Environment


[Installing Duckdb in Linux](https://duckdb.org/docs/installation/?version=stable&environment=cli&platform=linux&download_method=direct&architecture=x86_64)

[Installing Tippecanoe](https://github.com/ShogoHirasawa/vector-tile-maker/blob/main/vectorTileMaker.ipynb)


## ETL - Converting POI to PMTiles

[Reference](https://github.com/wipfli/foursquare-os-places-pmtiles/blob/main/run.sh)


To be added in the next query:

[FourSquare Categories](https://docs.foursquare.com/data-products/docs/categories#places-open-source--propremium-flat-file)

In [ ]:
%%shell

## Install duckdb CLI
curl https://install.duckdb.org | sh

## Install tippecanoe

# Clone the tippecanoe repository from GitHub
git clone https://github.com/mapbox/tippecanoe.git
# Enter the tippecanoe folder
cd tippecanoe
# Build tippecanoe using multiple cores (-j)
make -j
# Install tippecanoe in the system
make install
# Go back to the previous directory
cd ..

## Check if duckdb & tippecanoe are installed
echo $(/root/.duckdb/cli/latest/duckdb --version)
echo $(/content/tippecanoe/tippecanoe --version)

## Now Convert the Foursquare POI to pmtiles

set -e
set -u
/root/.duckdb/cli/latest/duckdb -c "
INSTALL spatial;
load spatial;

COPY (
    SELECT
        'Feature' AS type,
        json_object(
            'type', 'Point',
            'coordinates', [longitude, latitude]
        ) as geometry,
        json_object(
            'fsq_place_id', fsq_place_id,
            'name', name,
            'address', address,
            'locality', locality,
            'postcode', postcode,
            'admin_region', admin_region,
            'post_town', post_town,
            'country', country,
            'date_created', date_created,
            'date_refreshed', date_refreshed,
            'date_closed', date_closed,
            'website', website,
            'email', email,
            'fsq_category_ids', fsq_category_ids,
            'fsq_category_labels', fsq_category_labels,
            'dt', dt
        ) AS properties
    FROM read_parquet('s3://fsq-os-places-us-east-1/release/dt=2025-09-09/places/parquet/*')
    WHERE longitude IS NOT NULL AND latitude IS NOT NULL AND country = 'IN'
) TO STDOUT (FORMAT json);
" | /content/tippecanoe/tippecanoe -o foursquare-os-places-2025-09-09.pmtiles --force -l place -rg --drop-densest-as-needed --extend-zooms-if-still-dropping --maximum-tile-bytes=2500000 --progress-interval=10

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3517  100  3517    0     0   8476      0 --:--:-- --:--:-- --:--:--  8495

*** DuckDB Linux/MacOS installation script, version 1.4.0 ***


         .;odxdl,            
       .xXXXXXXXXKc          
       0XXXXXXXXXXXd  cooo:  
      ,XXXXXXXXXXXXK  OXXXXd 
       0XXXXXXXXXXXo  cooo:  
       .xXXXXXXXXKc          
         .;odxdl,  


######################################################################## 100.0%

  with a link from                      /root/.duckdb/cli/latest/duckdb

Hint: Append the following line to your shell profile:
export PATH='/root/.duckdb/cli/latest':$PATH


To launch DuckDB now, type
/root/.duckdb/cli/latest/duckdb
Cloning into 'tippecanoe'...
remote: Enumerating objects: 13569, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 1